#### 5.1 CNN 기초


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA=torch.cuda.is_available()
DEVICE=torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS=40
BATCH_SIZE=64

In [4]:
train_loader=torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                                                        transforms.ToTensor(),
                                                        transforms.Normalize((0.1307,),(0.3081,))
                          ])),
                          batch_size=BATCH_SIZE,shuffle=True
)
test_loader=torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          transform=transforms.Compose([
                                                         transforms.ToTensor(),
                                                         transforms.Normalize((0.1307,),(0.3081,))
                          ])),
                          batch_size=BATCH_SIZE,shuffle=True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1=nn.Conv2d(1,10,kernel_size=5)
    self.conv2=nn.Conv2d(10,20,kernel_size=5)
    self.drop=nn.Dropout2d()
    self.fc1=nn.Linear(320,50)
    self.fc2=nn.Linear(50,10)
  
  def forward(self,x):
    x=F.relu(F.max_pool2d(self.conv1(x),2))
    x=F.relu(F.max_pool2d(self.conv2(x),2))
    x=x.view(-1,320)# -1: 남는차원 모두, 320: x가 가진 원소의 개수
    x=F.relu(self.fc1(x))
    x=self.drop(x)
    x=self.fc2(x)
    return F.log_softmax(x,dim=1)

In [6]:
model=CNN().to(DEVICE)
optimizer=optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

In [7]:
def train(model,train_loader,optimizer,epoch):
  model.train()
  for batch_idx,(data,target) in enumerate(train_loader):
    data,target=data.to(DEVICE),target.to(DEVICE)
    optimizer.zero_grad()
    output=model(data)
    loss=F.cross_entropy(output,target)
    loss.backward()
    optimizer.step()

    if batch_idx%200==0:
      print('Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(epoch,batch_idx*len(data),
                                                                   len(train_loader.dataset),
                                                                   100. *batch_idx/len(train_loader),
                                                                   loss.item()))

In [8]:
def evaluate(model,test_loader):
  model.eval()
  test_loss=0
  correct=0
  with torch.no_grad():
    for data,target in test_loader:
      data,target=data.to(DEVICE),target.to(DEVICE)
      output=model(data)

      test_loss+=F.cross_entropy(output,target,reduction='sum').item()

      pred=output.max(1,keepdim=True)[1]
      correct+=pred.eq(target.view_as(pred)).sum().item()

  test_loss/=len(test_loader.dataset)
  test_accuracy=100.*correct/len(test_loader.dataset)
  return test_loss,test_accuracy

In [9]:
for epoch in range(1,EPOCHS+1):
  train(model,train_loader,optimizer,epoch)
  test_loss,test_accuracy=evaluate(model,test_loader)

  print('[{}] Test Loss: {:.4f}, Accuracy:{:.2f}%'.format(epoch,test_loss,test_accuracy))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch:1 [0/60000 (0%)]	Loss:2.292970
Train Epoch:1 [12800/60000 (21%)]	Loss:1.231801
Train Epoch:1 [25600/60000 (43%)]	Loss:0.762767
Train Epoch:1 [38400/60000 (64%)]	Loss:0.756676
Train Epoch:1 [51200/60000 (85%)]	Loss:0.820150
[1] Test Loss: 0.6153, Accuracy:76.47%
Train Epoch:2 [0/60000 (0%)]	Loss:0.818581
Train Epoch:2 [12800/60000 (21%)]	Loss:0.776829
Train Epoch:2 [25600/60000 (43%)]	Loss:0.651105
Train Epoch:2 [38400/60000 (64%)]	Loss:0.646008
Train Epoch:2 [51200/60000 (85%)]	Loss:0.539278
[2] Test Loss: 0.5237, Accuracy:79.79%
Train Epoch:3 [0/60000 (0%)]	Loss:0.488908
Train Epoch:3 [12800/60000 (21%)]	Loss:0.593795
Train Epoch:3 [25600/60000 (43%)]	Loss:0.542724
Train Epoch:3 [38400/60000 (64%)]	Loss:0.541043
Train Epoch:3 [51200/60000 (85%)]	Loss:0.346555
[3] Test Loss: 0.4717, Accuracy:82.78%
Train Epoch:4 [0/60000 (0%)]	Loss:0.485041
Train Epoch:4 [12800/60000 (21%)]	Loss:0.521651
Train Epoch:4 [25600/60000 (43%)]	Loss:0.523222
Train Epoch:4 [38400/60000 (64%)]	Loss: